In [1]:
"""
Set up the backtesting environment with all necessary imports.
This cell must run first.
"""

import sys
sys.path.append('../src')

# Auto-reload modules when they change
from IPython import get_ipython
ipython = get_ipython()
if ipython is not None:
    ipython.run_line_magic('load_ext', 'autoreload')
    ipython.run_line_magic('autoreload', '2')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from data_fetcher import StockDataFetcher
from indicators import (sma, ema, rsi, macd, bollinger_bands, 
                        adx, atr, stochastic, cci, mfi, cmf, 
                        supertrend, keltner_channels, roc, obv, vwap,
                        squeeze_momentum, williams_r)

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print('Backtesting Framework Setup Complete')
print('Ready to validate swing trading signals on historical data')



Extended indicators module loaded
Backtesting Framework Setup Complete
Ready to validate swing trading signals on historical data


In [2]:

# ============================================================================
# CELL 2: Core Indicator Calculation Function
# ============================================================================
"""
Reusable function to calculate all indicators on any dataframe.
This is the EXACT same calculation as Cell 3 from swing trading notebook.
"""

def calculate_indicators(df):
    """
    Calculate all technical indicators for backtesting.
    
    Args:
        df: DataFrame with OHLCV data
        
    Returns:
        DataFrame with all indicators added
    """
    # Make a copy to avoid modifying original
    data = df.copy()
    
    # ========================================
    # 1. TREND INDICATORS
    # ========================================
    data['SMA_20'] = sma(data['Close'], 20)
    data['SMA_50'] = sma(data['Close'], 50)
    data['SMA_200'] = sma(data['Close'], 200)
    data['EMA_12'] = ema(data['Close'], 12)
    data['EMA_26'] = ema(data['Close'], 26)
    
    # ADX with directional movement
    adx_result = adx(data['High'], data['Low'], data['Close'], window=14)
    data['ADX'] = adx_result['ADX']
    data['Plus_DI'] = adx_result['Plus_DI']
    data['Minus_DI'] = adx_result['Minus_DI']
    
    # SuperTrend
    st_result = supertrend(data['High'], data['Low'], data['Close'], atr_period=10, multiplier=3)
    data['SuperTrend'] = st_result['SuperTrend']
    data['SuperTrend_Direction'] = st_result['Direction']
    
    # ========================================
    # 2. MOMENTUM INDICATORS
    # ========================================
    data['RSI'] = rsi(data['Close'], window=14)
    data['CCI'] = cci(data['High'], data['Low'], data['Close'], window=20)
    data['ROC'] = roc(data['Close'], window=10)
    data['Williams_R'] = williams_r(data['High'], data['Low'], data['Close'], window=14)
    
    # Stochastic
    stoch_result = stochastic(data['High'], data['Low'], data['Close'], k_window=14, d_window=3)
    data['Stoch_K'] = stoch_result['K']
    data['Stoch_D'] = stoch_result['D']
    
    # MACD
    macd_result = macd(data['Close'], fast=12, slow=26, signal=9)
    data['MACD'] = macd_result['MACD']
    data['MACD_Signal'] = macd_result['Signal']
    data['MACD_Hist'] = macd_result['Histogram']
    
    # ========================================
    # 3. VOLATILITY INDICATORS
    # ========================================
    data['ATR'] = atr(data['High'], data['Low'], data['Close'], window=14)
    
    # Bollinger Bands
    bb_result = bollinger_bands(data['Close'], window=20, num_std=2)
    data['BB_Upper'] = bb_result['Upper']
    data['BB_Middle'] = bb_result['Middle']
    data['BB_Lower'] = bb_result['Lower']
    
    # Keltner Channels
    kc_result = keltner_channels(data['High'], data['Low'], data['Close'], window=20, atr_period=10, multiplier=2)
    data['KC_Upper'] = kc_result['Upper']
    data['KC_Middle'] = kc_result['Middle']
    data['KC_Lower'] = kc_result['Lower']
    
    # ========================================
    # 4. VOLUME INDICATORS
    # ========================================
    data['OBV'] = obv(data['Close'], data['Volume'])
    data['MFI'] = mfi(data['High'], data['Low'], data['Close'], data['Volume'], window=14)
    data['CMF'] = cmf(data['High'], data['Low'], data['Close'], data['Volume'], window=20)
    data['VWAP'] = vwap(data['High'], data['Low'], data['Close'], data['Volume'])
    
    # ========================================
    # 5. VOLUME ANALYSIS
    # ========================================
    data['Volume_SMA_20'] = sma(data['Volume'], 20)
    data['Volume_Ratio'] = data['Volume'] / data['Volume_SMA_20']
    
    # ========================================
    # 6. VOLATILITY REGIME
    # ========================================
    data['Returns'] = data['Close'].pct_change()
    data['Historical_Vol_20'] = data['Returns'].rolling(window=20).std() * np.sqrt(252)
    
    # Adaptive window for Vol_Percentile
    data_length = len(data)
    vol_window = min(data_length - 200, 120)
    if vol_window > 20:
        data['Vol_Percentile'] = data['Historical_Vol_20'].rolling(window=vol_window).apply(
            lambda x: pd.Series(x).rank(pct=True).iloc[-1]
        )
    else:
        data['Vol_Percentile'] = 0.5
    
    # ========================================
    # 7. PRICE STRUCTURE
    # ========================================
    data['Higher_High'] = (data['High'] > data['High'].shift(1)).astype(int)
    data['Lower_Low'] = (data['Low'] < data['Low'].shift(1)).astype(int)
    
    # ========================================
    # 8. SQUEEZE MOMENTUM
    # ========================================
    squeeze_result = squeeze_momentum(data['High'], data['Low'], data['Close'], bb_length=20, kc_length=20)
    data['Squeeze_On'] = squeeze_result['Squeeze_On']
    data['Squeeze_Momentum'] = squeeze_result['Momentum']
    
    # Drop rows with NaN values
    data = data.dropna()
    
    return data



In [ ]:
"""
Import scoring functions from swing trading system.
These are the EXACT same functions used for live analysis.
"""

def detect_market_regime(df):
    """
    Detect market regime using ADX and directional indicators.
    
    Returns:
        tuple: (trend, volatility, recommendation)
    """
    current = df.iloc[-1]
    
    # Trend Detection using ADX and DI
    adx_val = current['ADX']
    plus_di = current['Plus_DI']
    minus_di = current['Minus_DI']
    
    if adx_val > 20:
        if plus_di > minus_di:
            trend = 'UPTREND'
        else:
            trend = 'DOWNTREND'
    elif adx_val > 20:
        trend = 'TRANSITIONING'
    else:
        trend = 'RANGING'
    
    # Volatility Regime
    vol_pct = current['Vol_Percentile']
    if vol_pct > 0.7:
        volatility = 'HIGH_VOL'
    elif vol_pct < 0.3:
        volatility = 'LOW_VOL'
    else:
        volatility = 'NORMAL_VOL'
    
    # Strategy Recommendation
    if trend == 'UPTREND' and volatility == 'NORMAL_VOL':
        recommendation = 'Trend following on pullbacks to MA'
    elif trend == 'UPTREND' and volatility == 'HIGH_VOL':
        recommendation = 'Reduce size, wait for consolidation'
    elif trend == 'RANGING' and volatility == 'NORMAL_VOL':
        recommendation = 'Mean reversion (RSI extremes)'
    elif trend == 'RANGING' and volatility == 'HIGH_VOL':
        recommendation = 'Stay out, no clear edge'
    elif trend == 'DOWNTREND':
        recommendation = 'Reduce exposure or short bias'
    elif trend == 'TRANSITIONING':
        recommendation = 'Wait for trend confirmation'
    else:
        recommendation = 'Monitor for setup'
    
    return trend, volatility, recommendation


def calculate_improved_score(df):
    """
    Calculate composite technical score with proper weighting.
    Score range: -15 to +15
    
    Scoring Components:
    1. Trend (ADX + SuperTrend): ±5 points
    2. Momentum Confluence: ±3 to ±6 points
    3. Volume Confirmation: ±2 points
    4. Price Structure: ±2 to ±3 points
    5. VWAP Deviation: ±2 points
    6. Squeeze Detection: -2 points
    7. MACD: ±1 point
    8. Volatility Adjustment: ×0.7 if high vol
    """
    current = df.iloc[-1]
    score = 0
    
    # ========================================
    # 1. TREND STRENGTH (Highest Weight)
    # ========================================
    adx_val = current['ADX']
    plus_di = current['Plus_DI']
    minus_di = current['Minus_DI']
    
    if adx_val > 25:
        if plus_di > minus_di:
            score += 5
        else:
            score -= 5
    elif adx_val > 20:
        if plus_di > minus_di:
            score += 3
        else:
            score -= 3
    
    # ========================================
    # 2. MOMENTUM CONFLUENCE
    # ========================================
    oversold_count = 0
    overbought_count = 0
    
    if current['RSI'] < 40:
        oversold_count += 1
    elif current['RSI'] > 60:
        overbought_count += 1
    
    if current['Stoch_K'] < 20:
        oversold_count += 1
    elif current['Stoch_K'] > 80:
        overbought_count += 1
    
    if current['MFI'] < 20:
        oversold_count += 1
    elif current['MFI'] > 80:
        overbought_count += 1
    
    if current['CCI'] < -100:
        oversold_count += 1
    elif current['CCI'] > 100:
        overbought_count += 1
    
    # Non-linear scoring for confluence
    if oversold_count == 1:
        score += 3
    elif oversold_count == 2:
        score += 5
    elif oversold_count >= 3:
        score += 6
    
    if overbought_count == 1:
        score -= 3
    elif overbought_count == 2:
        score -= 5
    elif overbought_count >= 3:
        score -= 6
    
    # ========================================
    # 3. VOLUME CONFIRMATION
    # ========================================
    volume_spike = current['Volume_Ratio'] > 1.5
    cmf_positive = current['CMF'] > 0.05
    cmf_negative = current['CMF'] < -0.05
    
    if volume_spike and cmf_positive:
        score += 2
    elif volume_spike and cmf_negative:
        score -= 2
    
    # ========================================
    # 4. PRICE STRUCTURE
    # ========================================
    close_price = current['Close']
    sma_20 = current['SMA_20']
    sma_50 = current['SMA_50']
    
    if close_price > sma_20 > sma_50:
        score += 3
    elif close_price < sma_20 < sma_50:
        score -= 3
    elif close_price > sma_20:
        score += 2
    elif close_price < sma_20:
        score -= 2
    
    # ========================================
    # 5. VWAP DEVIATION (Mean Reversion)
    # ========================================
    vwap_val = current['VWAP']
    vwap_dev = (close_price - vwap_val) / vwap_val * 100
    
    if vwap_dev > 3:
        score -= 2
    elif vwap_dev < -3:
        score += 2
    
    # ========================================
    # 6. SQUEEZE DETECTION
    # ========================================
    if current['Squeeze_On']:
        score -= 2
    
    # ========================================
    # 7. MACD CONFIRMATION
    # ========================================
    if current['MACD'] > current['MACD_Signal']:
        score += 1
    elif current['MACD'] < current['MACD_Signal']:
        score -= 1
    
    # ========================================
    # 8. VOLATILITY ADJUSTMENT
    # ========================================
    if current['Vol_Percentile'] > 0.7:
        score = score * 0.7
    
    return round(score, 1)


def calculate_risk_reward(df):
    """
    Calculate risk/reward ratio for backtesting.
    
    Returns:
        dict: Entry, stop, target, R:R ratio
    """
    current = df.iloc[-1]
    
    # Entry price
    entry = current['Close']
    
    # Stop loss: Entry - (2 × ATR)
    atr_val = current['ATR']
    stop = entry - (2 * atr_val)
    
    # Target: 20-day high
    target = df['High'].tail(20).max()
    
    # Calculate R:R
    risk = entry - stop
    reward = target - entry
    
    if risk > 0:
        rr_ratio = reward / risk
    else:
        rr_ratio = 0
    
    return {
        'entry': entry,
        'stop': stop,
        'target': target,
        'risk': risk,
        'reward': reward,
        'rr_ratio': rr_ratio
    }


def generate_signal(score, rr_ratio):
    """
    Generate trading signal based on score and R:R ratio.
    
    Args:
        score: Technical score (-15 to +15)
        rr_ratio: Risk/Reward ratio
        
    Returns:
        str: Signal classification
    """
    if score >= 8 and rr_ratio >= 2.5:
        return 'STRONG BUY'
    elif score >= 6 and rr_ratio >= 2.0:
        return 'BUY'
    elif score <= -8 and rr_ratio >= 2.5:
        return 'STRONG SELL'
    elif score <= -6 and rr_ratio >= 2.0:
        return 'SELL'
    else:
        return 'NO TRADE'



In [4]:
"""
Main backtesting engine that simulates trades based on signals.
"""

def backtest_strategy(df, initial_capital=10000, risk_per_trade=0.01, 
                     signal_types=['BUY', 'STRONG BUY']):
    """
    Backtest the swing trading strategy on historical data.
    
    Args:
        df: DataFrame with all indicators calculated
        initial_capital: Starting account balance
        risk_per_trade: Percentage of account to risk per trade (0.01 = 1%)
        signal_types: Which signals to trade (e.g., ['BUY', 'STRONG BUY'])
        
    Returns:
        dict: Backtest results with trades and performance metrics
    """
    
    # Initialize tracking variables
    capital = initial_capital
    position = None  # Current open position
    trades = []  # List of completed trades
    equity_curve = []  # Track account value over time
    
    print(f'Starting Backtest')
    print(f'Initial Capital: ${initial_capital:,.2f}')
    print(f'Risk Per Trade: {risk_per_trade*100}%')
    print(f'Signal Types: {signal_types}')
    print('-' * 60)
    
    # Loop through each day
    for i in range(len(df)):
        current_date = df.index[i]
        current_data = df.iloc[:i+1]  # Data up to current date
        
        # Skip if not enough data for indicators
        if len(current_data) < 60:
            equity_curve.append({'Date': current_date, 'Equity': capital})
            continue
        
        # Get current price
        current_price = current_data.iloc[-1]['Close']
        
        # ========================================
        # CHECK EXIT CONDITIONS (if in position)
        # ========================================
        if position is not None:
            # Check stop loss
            if current_price <= position['stop']:
                # Stop loss hit
                exit_price = position['stop']
                pnl = (exit_price - position['entry']) * position['shares']
                pnl_pct = (exit_price / position['entry'] - 1) * 100
                
                capital += pnl
                
                trades.append({
                    'Entry_Date': position['entry_date'],
                    'Exit_Date': current_date,
                    'Entry_Price': position['entry'],
                    'Exit_Price': exit_price,
                    'Shares': position['shares'],
                    'PnL': pnl,
                    'PnL_Pct': pnl_pct,
                    'Exit_Reason': 'STOP_LOSS',
                    'Hold_Days': (current_date - position['entry_date']).days,
                    'Score': position['score'],
                    'Signal': position['signal']
                })
                
                position = None
                
            # Check target
            elif current_price >= position['target']:
                # Target hit
                exit_price = position['target']
                pnl = (exit_price - position['entry']) * position['shares']
                pnl_pct = (exit_price / position['entry'] - 1) * 100
                
                capital += pnl
                
                trades.append({
                    'Entry_Date': position['entry_date'],
                    'Exit_Date': current_date,
                    'Entry_Price': position['entry'],
                    'Exit_Price': exit_price,
                    'Shares': position['shares'],
                    'PnL': pnl,
                    'PnL_Pct': pnl_pct,
                    'Exit_Reason': 'TARGET',
                    'Hold_Days': (current_date - position['entry_date']).days,
                    'Score': position['score'],
                    'Signal': position['signal']
                })
                
                position = None
                
            # Check time-based exit (60 days max hold)
            elif (current_date - position['entry_date']).days >= 60:
                # Max hold period reached
                exit_price = current_price
                pnl = (exit_price - position['entry']) * position['shares']
                pnl_pct = (exit_price / position['entry'] - 1) * 100
                
                capital += pnl
                
                trades.append({
                    'Entry_Date': position['entry_date'],
                    'Exit_Date': current_date,
                    'Entry_Price': position['entry'],
                    'Exit_Price': exit_price,
                    'Shares': position['shares'],
                    'PnL': pnl,
                    'PnL_Pct': pnl_pct,
                    'Exit_Reason': 'TIME_EXIT',
                    'Hold_Days': (current_date - position['entry_date']).days,
                    'Score': position['score'],
                    'Signal': position['signal']
                })
                
                position = None
        
        # ========================================
        # CHECK ENTRY CONDITIONS (if no position)
        # ========================================
        if position is None:
            # Calculate score and signal
            score = calculate_improved_score(current_data)
            rr_data = calculate_risk_reward(current_data)
            signal = generate_signal(score, rr_data['rr_ratio'])
            
            # Check if signal matches our trading criteria
            if signal in signal_types:
                # Calculate position size
                risk_dollars = capital * risk_per_trade
                risk_per_share = rr_data['risk']
                
                if risk_per_share > 0:
                    shares = int(risk_dollars / risk_per_share)
                    
                    if shares > 0:
                        # Enter position
                        position = {
                            'entry_date': current_date,
                            'entry': rr_data['entry'],
                            'stop': rr_data['stop'],
                            'target': rr_data['target'],
                            'shares': shares,
                            'score': score,
                            'signal': signal
                        }
                        
                        # Deduct position cost from capital
                        capital -= (rr_data['entry'] * shares)
        
        # Track equity
        if position is not None:
            position_value = position['shares'] * current_price
            total_equity = capital + position_value
        else:
            total_equity = capital
        
        equity_curve.append({'Date': current_date, 'Equity': total_equity})
    
    # Close any remaining position at end of backtest
    if position is not None:
        exit_price = df.iloc[-1]['Close']
        pnl = (exit_price - position['entry']) * position['shares']
        pnl_pct = (exit_price / position['entry'] - 1) * 100
        
        capital += pnl
        
        trades.append({
            'Entry_Date': position['entry_date'],
            'Exit_Date': df.index[-1],
            'Entry_Price': position['entry'],
            'Exit_Price': exit_price,
            'Shares': position['shares'],
            'PnL': pnl,
            'PnL_Pct': pnl_pct,
            'Exit_Reason': 'END_OF_DATA',
            'Hold_Days': (df.index[-1] - position['entry_date']).days,
            'Score': position['score'],
            'Signal': position['signal']
        })
    
    # Convert to DataFrames
    trades_df = pd.DataFrame(trades)
    equity_df = pd.DataFrame(equity_curve)
    
    # Calculate performance metrics
    if len(trades_df) > 0:
        total_trades = len(trades_df)
        winning_trades = len(trades_df[trades_df['PnL'] > 0])
        losing_trades = len(trades_df[trades_df['PnL'] < 0])
        win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0
        
        total_pnl = trades_df['PnL'].sum()
        total_return_pct = ((capital / initial_capital) - 1) * 100
        
        avg_win = trades_df[trades_df['PnL'] > 0]['PnL'].mean() if winning_trades > 0 else 0
        avg_loss = trades_df[trades_df['PnL'] < 0]['PnL'].mean() if losing_trades > 0 else 0
        avg_win_pct = trades_df[trades_df['PnL'] > 0]['PnL_Pct'].mean() if winning_trades > 0 else 0
        avg_loss_pct = trades_df[trades_df['PnL'] < 0]['PnL_Pct'].mean() if losing_trades > 0 else 0
        
        # Profit factor
        gross_profit = trades_df[trades_df['PnL'] > 0]['PnL'].sum()
        gross_loss = abs(trades_df[trades_df['PnL'] < 0]['PnL'].sum())
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
        
        # Drawdown calculation
        equity_df['Peak'] = equity_df['Equity'].cummax()
        equity_df['Drawdown'] = (equity_df['Equity'] - equity_df['Peak']) / equity_df['Peak'] * 100
        max_drawdown = equity_df['Drawdown'].min()
        
        # Average hold time
        avg_hold_days = trades_df['Hold_Days'].mean()
        
        metrics = {
            'Total_Trades': total_trades,
            'Winning_Trades': winning_trades,
            'Losing_Trades': losing_trades,
            'Win_Rate': win_rate,
            'Total_PnL': total_pnl,
            'Total_Return_Pct': total_return_pct,
            'Avg_Win': avg_win,
            'Avg_Loss': avg_loss,
            'Avg_Win_Pct': avg_win_pct,
            'Avg_Loss_Pct': avg_loss_pct,
            'Profit_Factor': profit_factor,
            'Max_Drawdown': max_drawdown,
            'Final_Capital': capital,
            'Avg_Hold_Days': avg_hold_days
        }
    else:
        metrics = {
            'Total_Trades': 0,
            'Winning_Trades': 0,
            'Losing_Trades': 0,
            'Win_Rate': 0,
            'Total_PnL': 0,
            'Total_Return_Pct': 0,
            'Avg_Win': 0,
            'Avg_Loss': 0,
            'Avg_Win_Pct': 0,
            'Avg_Loss_Pct': 0,
            'Profit_Factor': 0,
            'Max_Drawdown': 0,
            'Final_Capital': initial_capital,
            'Avg_Hold_Days': 0
        }
    
    return {
        'trades': trades_df,
        'equity_curve': equity_df,
        'metrics': metrics
    }

In [5]:
"""
Run a backtest on a single stock and display results.
Change the ticker symbol to test different stocks.
"""

# Configuration
ticker = 'AAPL'  # Change this to any ticker
period = '5y'    # Use 5 years for thorough testing
initial_capital = 10000
risk_per_trade = 0.01  # 1% risk per trade
signal_types = ['BUY', 'STRONG BUY']  # Only trade high-quality signals

print(f'Backtesting {ticker} - Swing Trading Strategy')
print('=' * 60)

# Fetch data
fetcher = StockDataFetcher()
df = fetcher.fetch(ticker, period=period)

if df is not None and len(df) > 0:
    # Calculate indicators
    print('Calculating technical indicators...')
    df_with_indicators = calculate_indicators(df)
    
    # Run backtest
    print('Running backtest simulation...')
    results = backtest_strategy(
        df_with_indicators,
        initial_capital=initial_capital,
        risk_per_trade=risk_per_trade,
        signal_types=signal_types
    )
    
    # Display results
    print('\n' + '=' * 60)
    print('BACKTEST RESULTS')
    print('=' * 60)
    
    metrics = results['metrics']
    
    print(f"\nOVERALL PERFORMANCE")
    print(f"Initial Capital:    ${initial_capital:,.2f}")
    print(f"Final Capital:      ${metrics['Final_Capital']:,.2f}")
    print(f"Total Return:       {metrics['Total_Return_Pct']:.2f}%")
    print(f"Total PnL:          ${metrics['Total_PnL']:,.2f}")
    
    print(f"\nTRADE STATISTICS")
    print(f"Total Trades:       {metrics['Total_Trades']}")
    print(f"Winning Trades:     {metrics['Winning_Trades']}")
    print(f"Losing Trades:      {metrics['Losing_Trades']}")
    print(f"Win Rate:           {metrics['Win_Rate']:.1f}%")
    
    print(f"\nAVERAGE TRADE PERFORMANCE")
    print(f"Avg Win:            ${metrics['Avg_Win']:.2f} ({metrics['Avg_Win_Pct']:.2f}%)")
    print(f"Avg Loss:           ${metrics['Avg_Loss']:.2f} ({metrics['Avg_Loss_Pct']:.2f}%)")
    print(f"Profit Factor:      {metrics['Profit_Factor']:.2f}")
    print(f"Avg Hold Time:      {metrics['Avg_Hold_Days']:.1f} days")
    
    print(f"\nRISK METRICS")
    print(f"Max Drawdown:       {metrics['Max_Drawdown']:.2f}%")
    
    # Show sample trades
    if len(results['trades']) > 0:
        print('\n' + '=' * 60)
        print('SAMPLE TRADES (First 5)')
        print('=' * 60)
        print(results['trades'].head())
        
        print('\n' + '=' * 60)
        print('SAMPLE TRADES (Last 5)')
        print('=' * 60)
        print(results['trades'].tail())
else:
    print(f'Failed to fetch data for {ticker}')

Backtesting AAPL - Swing Trading Strategy
Fetching AAPL data...
Got 1255 days of data
Calculating technical indicators...
Running backtest simulation...
Starting Backtest
Initial Capital: $10,000.00
Risk Per Trade: 1.0%
Signal Types: ['BUY', 'STRONG BUY']
------------------------------------------------------------

BACKTEST RESULTS

OVERALL PERFORMANCE
Initial Capital:    $10,000.00
Final Capital:      $10,000.00
Total Return:       0.00%
Total PnL:          $0.00

TRADE STATISTICS
Total Trades:       0
Winning Trades:     0
Losing Trades:      0
Win Rate:           0.0%

AVERAGE TRADE PERFORMANCE
Avg Win:            $0.00 (0.00%)
Avg Loss:           $0.00 (0.00%)
Profit Factor:      0.00
Avg Hold Time:      0.0 days

RISK METRICS
Max Drawdown:       0.00%


In [6]:
"""
Create visualizations of backtest performance.
"""

if 'results' in locals() and len(results['trades']) > 0:
    fig, axes = plt.subplots(3, 1, figsize=(14, 12))
    
    # ========================================
    # Chart 1: Equity Curve
    # ========================================
    ax1 = axes[0]
    equity_data = results['equity_curve']
    ax1.plot(equity_data['Date'], equity_data['Equity'], linewidth=2, color='#2E86AB')
    ax1.axhline(y=initial_capital, color='gray', linestyle='--', alpha=0.7, label='Starting Capital')
    ax1.set_title(f'{ticker} - Equity Curve', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Account Value ($)', fontsize=11)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Format y-axis as currency
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    
    # ========================================
    # Chart 2: Drawdown
    # ========================================
    ax2 = axes[1]
    ax2.fill_between(equity_data['Date'], equity_data['Drawdown'], 0, 
                     color='#A23B72', alpha=0.5)
    ax2.plot(equity_data['Date'], equity_data['Drawdown'], 
            linewidth=2, color='#A23B72')
    ax2.set_title('Drawdown Analysis', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Drawdown (%)', fontsize=11)
    ax2.grid(True, alpha=0.3)
    
    # ========================================
    # Chart 3: Trade Distribution
    # ========================================
    ax3 = axes[2]
    trades_df = results['trades']
    
    # Create bins for PnL percentage
    bins = np.arange(-30, 35, 5)
    ax3.hist(trades_df['PnL_Pct'], bins=bins, color='#18A558', alpha=0.7, edgecolor='black')
    ax3.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Break-even')
    ax3.set_title('Trade Returns Distribution', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Return (%)', fontsize=11)
    ax3.set_ylabel('Number of Trades', fontsize=11)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save chart
    output_path = '../results/backtest_results.png'
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    print(f'\nChart saved to: {output_path}')
    
    plt.show()
else:
    print('No trades to visualize. Try different parameters or ticker.')

No trades to visualize. Try different parameters or ticker.


In [9]:


# ============================================================================
# CELL 6: Visualize Backtest Results
# ============================================================================
"""
Create visualizations of backtest performance.
"""

if 'results' in locals() and len(results['trades']) > 0:
    fig, axes = plt.subplots(3, 1, figsize=(14, 12))
    
    # ========================================
    # Chart 1: Equity Curve
    # ========================================
    ax1 = axes[0]
    equity_data = results['equity_curve']
    ax1.plot(equity_data['Date'], equity_data['Equity'], linewidth=2, color='#2E86AB')
    ax1.axhline(y=initial_capital, color='gray', linestyle='--', alpha=0.7, label='Starting Capital')
    ax1.set_title(f'{ticker} - Equity Curve', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Account Value ($)', fontsize=11)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Format y-axis as currency
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    
    # ========================================
    # Chart 2: Drawdown
    # ========================================
    ax2 = axes[1]
    ax2.fill_between(equity_data['Date'], equity_data['Drawdown'], 0, 
                     color='#A23B72', alpha=0.5)
    ax2.plot(equity_data['Date'], equity_data['Drawdown'], 
            linewidth=2, color='#A23B72')
    ax2.set_title('Drawdown Analysis', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Drawdown (%)', fontsize=11)
    ax2.grid(True, alpha=0.3)
    
    # ========================================
    # Chart 3: Trade Distribution
    # ========================================
    ax3 = axes[2]
    trades_df = results['trades']
    
    # Create bins for PnL percentage
    bins = np.arange(-30, 35, 5)
    ax3.hist(trades_df['PnL_Pct'], bins=bins, color='#18A558', alpha=0.7, edgecolor='black')
    ax3.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Break-even')
    ax3.set_title('Trade Returns Distribution', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Return (%)', fontsize=11)
    ax3.set_ylabel('Number of Trades', fontsize=11)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save chart
    output_path = '../results/backtest_results.png'
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    print(f'\nChart saved to: {output_path}')
    
    plt.show()
else:
    print('No trades to visualize. Try different parameters or ticker.')


# ============================================================================
# CELL 7: Multi-Stock Backtest Comparison
# ============================================================================
"""
Run backtests on multiple stocks and compare results.
This helps validate if the strategy works across different assets.
"""

# Stock list to test
test_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'NFLX']

# Configuration
period = '5y'
initial_capital = 10000
risk_per_trade = 0.01
signal_types = ['BUY', 'STRONG BUY']

print('Running Multi-Stock Backtest Comparison')
print('=' * 60)

# Store results for each ticker
all_results = {}

for ticker in test_tickers:
    print(f'\nTesting {ticker}...')
    
    try:
        # Fetch data
        fetcher = StockDataFetcher()
        df = fetcher.fetch(ticker, period=period)
        
        if df is not None and len(df) > 0:
            # Calculate indicators
            df_with_indicators = calculate_indicators(df)
            
            # Run backtest
            results = backtest_strategy(
                df_with_indicators,
                initial_capital=initial_capital,
                risk_per_trade=risk_per_trade,
                signal_types=signal_types
            )
            
            all_results[ticker] = results['metrics']
            print(f'{ticker}: {results["metrics"]["Total_Trades"]} trades, ' +
                  f'{results["metrics"]["Win_Rate"]:.1f}% win rate, ' +
                  f'{results["metrics"]["Total_Return_Pct"]:.2f}% return')
        else:
            print(f'{ticker}: Failed to fetch data')
            
    except Exception as e:
        print(f'{ticker}: Error - {e}')

# Create comparison DataFrame
if len(all_results) > 0:
    comparison_df = pd.DataFrame(all_results).T
    comparison_df = comparison_df.round(2)
    
    print('\n' + '=' * 60)
    print('MULTI-STOCK BACKTEST COMPARISON')
    print('=' * 60)
    print(comparison_df)
    
    # Summary statistics
    print('\n' + '=' * 60)
    print('STRATEGY SUMMARY STATISTICS')
    print('=' * 60)
    print(f"Average Win Rate:      {comparison_df['Win_Rate'].mean():.1f}%")
    print(f"Average Return:        {comparison_df['Total_Return_Pct'].mean():.2f}%")
    print(f"Average Profit Factor: {comparison_df['Profit_Factor'].mean():.2f}")
    print(f"Average Trades:        {comparison_df['Total_Trades'].mean():.0f}")
    print(f"Best Performer:        {comparison_df['Total_Return_Pct'].idxmax()} " +
          f"({comparison_df['Total_Return_Pct'].max():.2f}%)")
    print(f"Worst Performer:       {comparison_df['Total_Return_Pct'].idxmin()} " +
          f"({comparison_df['Total_Return_Pct'].min():.2f}%)")


No trades to visualize. Try different parameters or ticker.
Running Multi-Stock Backtest Comparison

Testing AAPL...
Fetching AAPL data...
Got 1255 days of data
Starting Backtest
Initial Capital: $10,000.00
Risk Per Trade: 1.0%
Signal Types: ['BUY', 'STRONG BUY']
------------------------------------------------------------
AAPL: 0 trades, 0.0% win rate, 0.00% return

Testing MSFT...
Fetching MSFT data...
Got 1255 days of data
Starting Backtest
Initial Capital: $10,000.00
Risk Per Trade: 1.0%
Signal Types: ['BUY', 'STRONG BUY']
------------------------------------------------------------
MSFT: 0 trades, 0.0% win rate, 0.00% return

Testing GOOGL...
Fetching GOOGL data...
Got 1255 days of data
Starting Backtest
Initial Capital: $10,000.00
Risk Per Trade: 1.0%
Signal Types: ['BUY', 'STRONG BUY']
------------------------------------------------------------
GOOGL: 1 trades, 0.0% win rate, -22.73% return

Testing AMZN...
Fetching AMZN data...
Got 1255 days of data
Starting Backtest
Initial C

In [16]:
# ============================================================================
# CELL 10: Small Cap Backtest Engine (Relaxed Thresholds + Fixed Targets)
# ============================================================================
"""
Modified backtest specifically for small cap swing trading.

KEY CHANGES FROM ORIGINAL:
1. Relaxed signal thresholds (Score ≥3 for BUY, ≥5 for STRONG BUY)
2. ATR-based targets (dynamic, always good R:R)
3. Adjusted R:R requirements (1.5:1 for BUY, 2.0:1 for STRONG BUY)
4. Small cap focused (expects higher volatility)
"""

def calculate_risk_reward_fixed(df, atr_multiplier_stop=2.0, atr_multiplier_target=4.0):
    """
    Calculate risk/reward with ATR-based targets (FIXED VERSION)
    
    Args:
        df: DataFrame with indicators
        atr_multiplier_stop: Stop loss distance (default 2× ATR)
        atr_multiplier_target: Target distance (default 4× ATR)
        
    Returns:
        dict: Entry, stop, target, R:R ratio
    """
    current = df.iloc[-1]
    
    # Entry price
    entry = current['Close']
    
    # ATR-based stop and target
    atr_val = current['ATR']
    stop = entry - (atr_multiplier_stop * atr_val)
    target = entry + (atr_multiplier_target * atr_val)
    
    # Calculate R:R
    risk = entry - stop
    reward = target - entry
    
    if risk > 0:
        rr_ratio = reward / risk
    else:
        rr_ratio = 0
    
    return {
        'entry': entry,
        'stop': stop,
        'target': target,
        'risk': risk,
        'reward': reward,
        'rr_ratio': rr_ratio
    }


def generate_signal_relaxed(score, rr_ratio):
    """
    Generate trading signal with RELAXED thresholds for small caps.
    
    CHANGES FROM ORIGINAL:
    - BUY: Score ≥3 (was 6), R:R ≥1.5 (was 2.0)
    - STRONG BUY: Score ≥5 (was 8), R:R ≥2.0 (was 2.5)
    
    Args:
        score: Technical score (-15 to +15)
        rr_ratio: Risk/Reward ratio
        
    Returns:
        str: Signal classification
    """
    if score >= 5 and rr_ratio >= 2.0:
        return 'STRONG BUY'
    elif score >= 3 and rr_ratio >= 1.5:
        return 'BUY'
    elif score <= -5 and rr_ratio >= 2.0:
        return 'STRONG SELL'
    elif score <= -3 and rr_ratio >= 1.5:
        return 'SELL'
    else:
        return 'NO TRADE'


def backtest_smallcap_strategy(df, initial_capital=10000, risk_per_trade=0.01, 
                               signal_types=['BUY', 'STRONG BUY'],
                               atr_stop=2.0, atr_target=4.0,
                               max_hold_days=60):
    """
    Backtest swing trading strategy optimized for SMALL CAPS.
    
    KEY DIFFERENCES:
    - Uses ATR-based targets (not 20-day high)
    - Relaxed signal thresholds
    - Expects higher volatility
    
    Args:
        df: DataFrame with all indicators calculated
        initial_capital: Starting account balance
        risk_per_trade: Percentage of account to risk per trade (0.01 = 1%)
        signal_types: Which signals to trade (e.g., ['BUY', 'STRONG BUY'])
        atr_stop: Stop loss multiplier (default 2× ATR)
        atr_target: Target multiplier (default 4× ATR)
        max_hold_days: Maximum days to hold position
        
    Returns:
        dict: Backtest results with trades and performance metrics
    """
    
    # Initialize tracking variables
    capital = initial_capital
    position = None  # Current open position
    trades = []  # List of completed trades
    equity_curve = []  # Track account value over time
    
    print(f'Starting Small Cap Backtest')
    print(f'Initial Capital: ${initial_capital:,.2f}')
    print(f'Risk Per Trade: {risk_per_trade*100}%')
    print(f'Signal Types: {signal_types}')
    print(f'Stop: {atr_stop}× ATR | Target: {atr_target}× ATR')
    print('-' * 60)
    
    # Loop through each day
    for i in range(len(df)):
        current_date = df.index[i]
        current_data = df.iloc[:i+1]  # Data up to current date
        
        # Skip if not enough data for indicators
        if len(current_data) < 60:
            equity_curve.append({'Date': current_date, 'Equity': capital})
            continue
        
        # Get current price
        current_price = current_data.iloc[-1]['Close']
        
        # ========================================
        # CHECK EXIT CONDITIONS (if in position)
        # ========================================
        if position is not None:
            # Check stop loss
            if current_price <= position['stop']:
                # Stop loss hit
                exit_price = position['stop']
                pnl = (exit_price - position['entry']) * position['shares']
                pnl_pct = (exit_price / position['entry'] - 1) * 100
                
                capital += pnl
                
                trades.append({
                    'Entry_Date': position['entry_date'],
                    'Exit_Date': current_date,
                    'Entry_Price': position['entry'],
                    'Exit_Price': exit_price,
                    'Shares': position['shares'],
                    'PnL': pnl,
                    'PnL_Pct': pnl_pct,
                    'Exit_Reason': 'STOP_LOSS',
                    'Hold_Days': (current_date - position['entry_date']).days,
                    'Score': position['score'],
                    'Signal': position['signal']
                })
                
                position = None
                
            # Check target
            elif current_price >= position['target']:
                # Target hit
                exit_price = position['target']
                pnl = (exit_price - position['entry']) * position['shares']
                pnl_pct = (exit_price / position['entry'] - 1) * 100
                
                capital += pnl
                
                trades.append({
                    'Entry_Date': position['entry_date'],
                    'Exit_Date': current_date,
                    'Entry_Price': position['entry'],
                    'Exit_Price': exit_price,
                    'Shares': position['shares'],
                    'PnL': pnl,
                    'PnL_Pct': pnl_pct,
                    'Exit_Reason': 'TARGET',
                    'Hold_Days': (current_date - position['entry_date']).days,
                    'Score': position['score'],
                    'Signal': position['signal']
                })
                
                position = None
                
            # Check time-based exit
            elif (current_date - position['entry_date']).days >= max_hold_days:
                # Max hold period reached
                exit_price = current_price
                pnl = (exit_price - position['entry']) * position['shares']
                pnl_pct = (exit_price / position['entry'] - 1) * 100
                
                capital += pnl
                
                trades.append({
                    'Entry_Date': position['entry_date'],
                    'Exit_Date': current_date,
                    'Entry_Price': position['entry'],
                    'Exit_Price': exit_price,
                    'Shares': position['shares'],
                    'PnL': pnl,
                    'PnL_Pct': pnl_pct,
                    'Exit_Reason': 'TIME_EXIT',
                    'Hold_Days': (current_date - position['entry_date']).days,
                    'Score': position['score'],
                    'Signal': position['signal']
                })
                
                position = None
        
        # ========================================
        # CHECK ENTRY CONDITIONS (if no position)
        # ========================================
        if position is None:
            # Calculate score and signal
            score = calculate_improved_score(current_data)
            rr_data = calculate_risk_reward_fixed(current_data, atr_stop, atr_target)
            signal = generate_signal_relaxed(score, rr_data['rr_ratio'])
            
            # Check if signal matches our trading criteria
            if signal in signal_types:
                # Calculate position size
                risk_dollars = capital * risk_per_trade
                risk_per_share = rr_data['risk']
                
                if risk_per_share > 0:
                    shares = int(risk_dollars / risk_per_share)
                    
                    if shares > 0:
                        # Enter position
                        position = {
                            'entry_date': current_date,
                            'entry': rr_data['entry'],
                            'stop': rr_data['stop'],
                            'target': rr_data['target'],
                            'shares': shares,
                            'score': score,
                            'signal': signal
                        }
                        
                        # Deduct position cost from capital
                        capital -= (rr_data['entry'] * shares)
        
        # Track equity
        if position is not None:
            position_value = position['shares'] * current_price
            total_equity = capital + position_value
        else:
            total_equity = capital
        
        equity_curve.append({'Date': current_date, 'Equity': total_equity})
    
    # Close any remaining position at end of backtest
    if position is not None:
        exit_price = df.iloc[-1]['Close']
        pnl = (exit_price - position['entry']) * position['shares']
        pnl_pct = (exit_price / position['entry'] - 1) * 100
        
        capital += pnl
        
        trades.append({
            'Entry_Date': position['entry_date'],
            'Exit_Date': df.index[-1],
            'Entry_Price': position['entry'],
            'Exit_Price': exit_price,
            'Shares': position['shares'],
            'PnL': pnl,
            'PnL_Pct': pnl_pct,
            'Exit_Reason': 'END_OF_DATA',
            'Hold_Days': (df.index[-1] - position['entry_date']).days,
            'Score': position['score'],
            'Signal': position['signal']
        })
    
    # Convert to DataFrames
    trades_df = pd.DataFrame(trades)
    equity_df = pd.DataFrame(equity_curve)
    
    # Calculate performance metrics
    if len(trades_df) > 0:
        total_trades = len(trades_df)
        winning_trades = len(trades_df[trades_df['PnL'] > 0])
        losing_trades = len(trades_df[trades_df['PnL'] < 0])
        win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0
        
        total_pnl = trades_df['PnL'].sum()
        total_return_pct = ((capital / initial_capital) - 1) * 100
        
        avg_win = trades_df[trades_df['PnL'] > 0]['PnL'].mean() if winning_trades > 0 else 0
        avg_loss = trades_df[trades_df['PnL'] < 0]['PnL'].mean() if losing_trades > 0 else 0
        avg_win_pct = trades_df[trades_df['PnL'] > 0]['PnL_Pct'].mean() if winning_trades > 0 else 0
        avg_loss_pct = trades_df[trades_df['PnL'] < 0]['PnL_Pct'].mean() if losing_trades > 0 else 0
        
        # Profit factor
        gross_profit = trades_df[trades_df['PnL'] > 0]['PnL'].sum()
        gross_loss = abs(trades_df[trades_df['PnL'] < 0]['PnL'].sum())
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
        
        # Drawdown calculation
        equity_df['Peak'] = equity_df['Equity'].cummax()
        equity_df['Drawdown'] = (equity_df['Equity'] - equity_df['Peak']) / equity_df['Peak'] * 100
        max_drawdown = equity_df['Drawdown'].min()
        
        # Average hold time
        avg_hold_days = trades_df['Hold_Days'].mean()
        
        # Exit reason breakdown
        exit_reasons = trades_df['Exit_Reason'].value_counts()
        
        metrics = {
            'Total_Trades': total_trades,
            'Winning_Trades': winning_trades,
            'Losing_Trades': losing_trades,
            'Win_Rate': win_rate,
            'Total_PnL': total_pnl,
            'Total_Return_Pct': total_return_pct,
            'Avg_Win': avg_win,
            'Avg_Loss': avg_loss,
            'Avg_Win_Pct': avg_win_pct,
            'Avg_Loss_Pct': avg_loss_pct,
            'Profit_Factor': profit_factor,
            'Max_Drawdown': max_drawdown,
            'Final_Capital': capital,
            'Avg_Hold_Days': avg_hold_days,
            'Exit_Reasons': exit_reasons.to_dict()
        }
    else:
        metrics = {
            'Total_Trades': 0,
            'Winning_Trades': 0,
            'Losing_Trades': 0,
            'Win_Rate': 0,
            'Total_PnL': 0,
            'Total_Return_Pct': 0,
            'Avg_Win': 0,
            'Avg_Loss': 0,
            'Avg_Win_Pct': 0,
            'Avg_Loss_Pct': 0,
            'Profit_Factor': 0,
            'Max_Drawdown': 0,
            'Final_Capital': initial_capital,
            'Avg_Hold_Days': 0,
            'Exit_Reasons': {}
        }
    
    return {
        'trades': trades_df,
        'equity_curve': equity_df,
        'metrics': metrics
    }


print('Small Cap Backtest Engine Loaded')
print('Key Changes:')
print('  - BUY threshold: Score >= 3, R:R >= 1.5')
print('  - STRONG BUY threshold: Score >= 5, R:R >= 2.0')
print('  - Targets: ATR-based (4× ATR target, 2× ATR stop)')
print('  - Expected: 2:1 R:R minimum')

Small Cap Backtest Engine Loaded
Key Changes:
  - BUY threshold: Score >= 3, R:R >= 1.5
  - STRONG BUY threshold: Score >= 5, R:R >= 2.0
  - Targets: ATR-based (4× ATR target, 2× ATR stop)
  - Expected: 2:1 R:R minimum


In [23]:
# ============================================================================
# CELL 12: FIXED SWING TRADING BACKTEST SYSTEM
# ============================================================================
"""
COMPLETE REWRITE with critical fixes:
1. MANDATORY trend filter (no trading in downtrends)
2. Better ticker selection (diversified, quality stocks)
3. Enhanced analysis and comparison

This replaces Cell 10 and Cell 11 entirely.
"""

import numpy as np
import pandas as pd

# ============================================================================
# SECTION 1: TREND FILTER (CRITICAL FIX)
# ============================================================================

def is_in_uptrend(df, lookback=60, strict=True):
    """
    MANDATORY trend filter - prevents trading downtrending stocks.
    This is THE critical fix that changes everything.
    
    Args:
        df: DataFrame with indicators
        lookback: Days to check for trend (default 60)
        strict: If True, all conditions must pass (recommended)
        
    Returns:
        bool: True if stock is in valid uptrend
    """
    current = df.iloc[-1]
    
    # Condition 1: Price above key moving averages
    above_sma20 = current['Close'] > current['SMA_20']
    above_sma50 = current['Close'] > current['SMA_50']
    
    # Condition 2: Moving averages in bullish alignment
    sma_aligned = current['SMA_20'] > current['SMA_50']
    
    # Condition 3: Making higher highs (not just sideways)
    recent_high = df['High'].tail(lookback).max()
    current_high = df['High'].tail(10).max()
    making_higher_highs = current_high >= (recent_high * 0.95)  # Within 5% of recent high
    
    # Condition 4: Not in steep decline (crash protection)
    price_change_pct = (current['Close'] / df['Close'].iloc[-lookback] - 1) * 100
    not_crashing = price_change_pct > -20  # Not down more than 20% over lookback
    
    # Condition 5: Positive momentum (optional but recommended)
    adx_strong = current['ADX'] > 20  # Some trend strength
    plus_di_bullish = current['Plus_DI'] > current['Minus_DI']
    
    if strict:
        # ALL conditions must be true
        return (above_sma20 and above_sma50 and sma_aligned and 
                making_higher_highs and not_crashing and 
                adx_strong and plus_di_bullish)
    else:
        # Relaxed: Most conditions true
        conditions_met = sum([
            above_sma20, above_sma50, sma_aligned, 
            making_higher_highs, not_crashing
        ])
        return conditions_met >= 4  # At least 4 out of 5


def calculate_risk_reward_fixed(df, atr_multiplier_stop=2.0, atr_multiplier_target=4.0):
    """
    Calculate risk/reward with ATR-based targets.
    Same as before - this part was working fine.
    """
    current = df.iloc[-1]
    
    entry = current['Close']
    atr_val = current['ATR']
    stop = entry - (atr_multiplier_stop * atr_val)
    target = entry + (atr_multiplier_target * atr_val)
    
    risk = entry - stop
    reward = target - entry
    
    if risk > 0:
        rr_ratio = reward / risk
    else:
        rr_ratio = 0
    
    return {
        'entry': entry,
        'stop': stop,
        'target': target,
        'risk': risk,
        'reward': reward,
        'rr_ratio': rr_ratio
    }


def generate_signal_with_trend_filter(df, score, rr_ratio, require_uptrend=True):
    """
    Generate signal with MANDATORY trend check.
    This is the key change - no signal without uptrend.
    
    Args:
        df: DataFrame with indicators
        score: Technical score
        rr_ratio: Risk/reward ratio
        require_uptrend: If True, force uptrend check (RECOMMENDED)
        
    Returns:
        str: Signal classification
    """
    # CRITICAL: Check trend first, before anything else
    if require_uptrend and not is_in_uptrend(df, strict=True):
        return 'NO TRADE'  # Exit immediately if not in uptrend
    
    # Now check score and R:R (same thresholds as before)
    if score >= 5 and rr_ratio >= 2.0:
        return 'STRONG BUY'
    elif score >= 3 and rr_ratio >= 1.5:
        return 'BUY'
    elif score <= -5 and rr_ratio >= 2.0:
        return 'STRONG SELL'
    elif score <= -3 and rr_ratio >= 1.5:
        return 'SELL'
    else:
        return 'NO TRADE'


# ============================================================================
# SECTION 2: IMPROVED BACKTEST ENGINE
# ============================================================================

def backtest_with_trend_filter(df, initial_capital=10000, risk_per_trade=0.01, 
                                signal_types=['BUY', 'STRONG BUY'],
                                atr_stop=2.0, atr_target=4.0,
                                max_hold_days=60,
                                require_uptrend=True):
    """
    Enhanced backtest with mandatory trend filtering.
    
    KEY CHANGE: Only enters trades when stock is in confirmed uptrend.
    """
    
    capital = initial_capital
    position = None
    trades = []
    equity_curve = []
    trend_checks = {'passed': 0, 'failed': 0}  # Track how many setups rejected
    
    print(f'Starting Enhanced Backtest with Trend Filter')
    print(f'Initial Capital: ${initial_capital:,.2f}')
    print(f'Risk Per Trade: {risk_per_trade*100}%')
    print(f'Signal Types: {signal_types}')
    print(f'Trend Filter: {"ENABLED (Strict)" if require_uptrend else "DISABLED"}')
    print(f'Stop: {atr_stop}× ATR | Target: {atr_target}× ATR')
    print('-' * 60)
    
    # Loop through each day
    for i in range(len(df)):
        current_date = df.index[i]
        current_data = df.iloc[:i+1]
        
        if len(current_data) < 60:
            equity_curve.append({'Date': current_date, 'Equity': capital})
            continue
        
        current_price = current_data.iloc[-1]['Close']
        
        # ========================================
        # EXIT LOGIC (Same as before)
        # ========================================
        if position is not None:
            # Check stop loss
            if current_price <= position['stop']:
                exit_price = position['stop']
                pnl = (exit_price - position['entry']) * position['shares']
                pnl_pct = (exit_price / position['entry'] - 1) * 100
                capital += pnl
                
                trades.append({
                    'Entry_Date': position['entry_date'],
                    'Exit_Date': current_date,
                    'Entry_Price': position['entry'],
                    'Exit_Price': exit_price,
                    'Shares': position['shares'],
                    'PnL': pnl,
                    'PnL_Pct': pnl_pct,
                    'Exit_Reason': 'STOP_LOSS',
                    'Hold_Days': (current_date - position['entry_date']).days,
                    'Score': position['score'],
                    'Signal': position['signal']
                })
                position = None
                
            # Check target
            elif current_price >= position['target']:
                exit_price = position['target']
                pnl = (exit_price - position['entry']) * position['shares']
                pnl_pct = (exit_price / position['entry'] - 1) * 100
                capital += pnl
                
                trades.append({
                    'Entry_Date': position['entry_date'],
                    'Exit_Date': current_date,
                    'Entry_Price': position['entry'],
                    'Exit_Price': exit_price,
                    'Shares': position['shares'],
                    'PnL': pnl,
                    'PnL_Pct': pnl_pct,
                    'Exit_Reason': 'TARGET',
                    'Hold_Days': (current_date - position['entry_date']).days,
                    'Score': position['score'],
                    'Signal': position['signal']
                })
                position = None
                
            # Check time exit
            elif (current_date - position['entry_date']).days >= max_hold_days:
                exit_price = current_price
                pnl = (exit_price - position['entry']) * position['shares']
                pnl_pct = (exit_price / position['entry'] - 1) * 100
                capital += pnl
                
                trades.append({
                    'Entry_Date': position['entry_date'],
                    'Exit_Date': current_date,
                    'Entry_Price': position['entry'],
                    'Exit_Price': exit_price,
                    'Shares': position['shares'],
                    'PnL': pnl,
                    'PnL_Pct': pnl_pct,
                    'Exit_Reason': 'TIME_EXIT',
                    'Hold_Days': (current_date - position['entry_date']).days,
                    'Score': position['score'],
                    'Signal': position['signal']
                })
                position = None
        
        # ========================================
        # ENTRY LOGIC (WITH TREND FILTER)
        # ========================================
        if position is None:
            # Calculate score
            score = calculate_improved_score(current_data)
            rr_data = calculate_risk_reward_fixed(current_data, atr_stop, atr_target)
            
            # NEW: Generate signal with trend check
            signal = generate_signal_with_trend_filter(
                current_data, score, rr_data['rr_ratio'], require_uptrend
            )
            
            # Track trend filter effectiveness
            if score >= 3 and rr_data['rr_ratio'] >= 1.5:
                # This would have been a signal without trend filter
                if signal in signal_types:
                    trend_checks['passed'] += 1
                else:
                    trend_checks['failed'] += 1  # Rejected by trend filter
            
            # Enter position only if signal matches
            if signal in signal_types:
                risk_dollars = capital * risk_per_trade
                risk_per_share = rr_data['risk']
                
                if risk_per_share > 0:
                    shares = int(risk_dollars / risk_per_share)
                    
                    if shares > 0:
                        position = {
                            'entry_date': current_date,
                            'entry': rr_data['entry'],
                            'stop': rr_data['stop'],
                            'target': rr_data['target'],
                            'shares': shares,
                            'score': score,
                            'signal': signal
                        }
                        capital -= (rr_data['entry'] * shares)
        
        # Track equity
        if position is not None:
            position_value = position['shares'] * current_price
            total_equity = capital + position_value
        else:
            total_equity = capital
        
        equity_curve.append({'Date': current_date, 'Equity': total_equity})
    
    # Close remaining position
    if position is not None:
        exit_price = df.iloc[-1]['Close']
        pnl = (exit_price - position['entry']) * position['shares']
        pnl_pct = (exit_price / position['entry'] - 1) * 100
        capital += pnl
        
        trades.append({
            'Entry_Date': position['entry_date'],
            'Exit_Date': df.index[-1],
            'Entry_Price': position['entry'],
            'Exit_Price': exit_price,
            'Shares': position['shares'],
            'PnL': pnl,
            'PnL_Pct': pnl_pct,
            'Exit_Reason': 'END_OF_DATA',
            'Hold_Days': (df.index[-1] - position['entry_date']).days,
            'Score': position['score'],
            'Signal': position['signal']
        })
    
    # Convert to DataFrames
    trades_df = pd.DataFrame(trades)
    equity_df = pd.DataFrame(equity_curve)
    
    # Calculate metrics
    if len(trades_df) > 0:
        total_trades = len(trades_df)
        winning_trades = len(trades_df[trades_df['PnL'] > 0])
        losing_trades = len(trades_df[trades_df['PnL'] < 0])
        win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0
        
        total_pnl = trades_df['PnL'].sum()
        total_return_pct = ((capital / initial_capital) - 1) * 100
        
        avg_win = trades_df[trades_df['PnL'] > 0]['PnL'].mean() if winning_trades > 0 else 0
        avg_loss = trades_df[trades_df['PnL'] < 0]['PnL'].mean() if losing_trades > 0 else 0
        avg_win_pct = trades_df[trades_df['PnL'] > 0]['PnL_Pct'].mean() if winning_trades > 0 else 0
        avg_loss_pct = trades_df[trades_df['PnL'] < 0]['PnL_Pct'].mean() if losing_trades > 0 else 0
        
        gross_profit = trades_df[trades_df['PnL'] > 0]['PnL'].sum()
        gross_loss = abs(trades_df[trades_df['PnL'] < 0]['PnL'].sum())
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
        
        equity_df['Peak'] = equity_df['Equity'].cummax()
        equity_df['Drawdown'] = (equity_df['Equity'] - equity_df['Peak']) / equity_df['Peak'] * 100
        max_drawdown = equity_df['Drawdown'].min()
        
        avg_hold_days = trades_df['Hold_Days'].mean()
        exit_reasons = trades_df['Exit_Reason'].value_counts()
        
        metrics = {
            'Total_Trades': total_trades,
            'Winning_Trades': winning_trades,
            'Losing_Trades': losing_trades,
            'Win_Rate': win_rate,
            'Total_PnL': total_pnl,
            'Total_Return_Pct': total_return_pct,
            'Avg_Win': avg_win,
            'Avg_Loss': avg_loss,
            'Avg_Win_Pct': avg_win_pct,
            'Avg_Loss_Pct': avg_loss_pct,
            'Profit_Factor': profit_factor,
            'Max_Drawdown': max_drawdown,
            'Final_Capital': capital,
            'Avg_Hold_Days': avg_hold_days,
            'Exit_Reasons': exit_reasons.to_dict(),
            'Trend_Checks': trend_checks
        }
    else:
        metrics = {
            'Total_Trades': 0,
            'Winning_Trades': 0,
            'Losing_Trades': 0,
            'Win_Rate': 0,
            'Total_PnL': 0,
            'Total_Return_Pct': 0,
            'Avg_Win': 0,
            'Avg_Loss': 0,
            'Avg_Win_Pct': 0,
            'Avg_Loss_Pct': 0,
            'Profit_Factor': 0,
            'Max_Drawdown': 0,
            'Final_Capital': initial_capital,
            'Avg_Hold_Days': 0,
            'Exit_Reasons': {},
            'Trend_Checks': trend_checks
        }
    
    return {
        'trades': trades_df,
        'equity_curve': equity_df,
        'metrics': metrics
    }


# ============================================================================
# SECTION 3: BETTER TICKER LISTS
# ============================================================================

# Healthcare & Biotech (moderate volatility, growth)
HEALTHCARE = ['HIMS', 'TDOC', 'ONEM', 'DOCS', 'ACCD']

# FinTech (tech-enabled finance)
FINTECH = ['SOFI', 'AFRM', 'UPST', 'HOOD', 'NU']

# Cloud & SaaS (high growth, profitable)
CLOUD_SAAS = ['DDOG', 'NET', 'MDB', 'ZS', 'CFLT']

# Consumer & E-commerce
CONSUMER = ['DASH', 'ABNB', 'RBLX', 'ETSY', 'PINS']

# Semiconductors & Hardware (cyclical)
SEMIS = ['ONTO', 'SLAB', 'SYNA', 'SWKS', 'QRVO']

# Mixed Best-of-Breed (RECOMMENDED FOR TESTING)
MIXED_QUALITY = [
    'HIMS', 'DOCS', 'TDOC',      # Healthcare
    'SOFI', 'AFRM',              # FinTech
    'DDOG', 'NET', 'MDB',        # Cloud/SaaS
    'DASH', 'ABNB',              # Consumer
    'ONTO', 'SLAB'               # Semis
]

# Old list (for comparison - DO NOT USE for real testing)
OLD_CRYPTO_EV = ['HIVE', 'MARA', 'RIOT', 'CLSK', 'BTBT', 'LCID', 'RIVN', 
                 'QS', 'CHPT', 'BLNK', 'EVGO', 'SOFI', 'UPST']


# ============================================================================
# SECTION 4: MULTI-STOCK BACKTEST WITH COMPARISON
# ============================================================================

def run_multi_stock_backtest(ticker_list, period='5y', initial_capital=10000,
                             risk_per_trade=0.01, signal_types=['BUY', 'STRONG BUY'],
                             require_uptrend=True, list_name='Custom'):
    """
    Run backtest on multiple stocks with enhanced analysis.
    """
    
    print('=' * 70)
    print(f'ENHANCED BACKTEST: {list_name}')
    print('=' * 70)
    print(f'Testing {len(ticker_list)} stocks')
    print(f'Period: {period}')
    print(f'Trend Filter: {"ENABLED" if require_uptrend else "DISABLED"}')
    print(f'Signal Thresholds: BUY (Score≥3, R:R≥1.5), STRONG BUY (Score≥5, R:R≥2.0)')
    print('=' * 70)
    
    all_results = {}
    all_trades = []
    
    for idx, ticker in enumerate(ticker_list):
        print(f'\n[{idx+1}/{len(ticker_list)}] Testing {ticker}...')
        
        try:
            fetcher = StockDataFetcher()
            df = fetcher.fetch(ticker, period=period)
            
            if df is not None and len(df) > 0:
                df_with_indicators = calculate_indicators(df)
                
                results = backtest_with_trend_filter(
                    df_with_indicators,
                    initial_capital=initial_capital,
                    risk_per_trade=risk_per_trade,
                    signal_types=signal_types,
                    atr_stop=2.0,
                    atr_target=4.0,
                    max_hold_days=60,
                    require_uptrend=require_uptrend
                )
                
                all_results[ticker] = results['metrics']
                
                if len(results['trades']) > 0:
                    trades_with_ticker = results['trades'].copy()
                    trades_with_ticker['Ticker'] = ticker
                    all_trades.append(trades_with_ticker)
                
                m = results['metrics']
                trend_info = m['Trend_Checks']
                print(f"  → {m['Total_Trades']} trades | " +
                      f"{m['Win_Rate']:.1f}% WR | " +
                      f"{m['Total_Return_Pct']:+.1f}% return | " +
                      f"PF: {m['Profit_Factor']:.2f}")
                print(f"     Trend Filter: Passed {trend_info['passed']}, " +
                      f"Rejected {trend_info['failed']} setups")
            else:
                print(f'  → Failed to fetch data')
                
        except Exception as e:
            print(f'  → Error: {str(e)[:60]}')
    
    return all_results, all_trades


# ============================================================================
# SECTION 5: RUN THE TEST
# ============================================================================

# CONFIGURATION - CHANGE THESE
test_list = ['HIMS']  # ← Change this to test different lists
list_name = 'Mixed Quality (Recommended)'
period = '2y'
initial_capital = 10000
risk_per_trade = 0.01
signal_types = ['BUY', 'STRONG BUY']
require_uptrend = True  # ← KEEP THIS TRUE!

# Run backtest
results, trades = run_multi_stock_backtest(
    ticker_list=test_list,
    period=period,
    initial_capital=initial_capital,
    risk_per_trade=risk_per_trade,
    signal_types=signal_types,
    require_uptrend=require_uptrend,
    list_name=list_name
)

# ============================================================================
# SECTION 6: RESULTS ANALYSIS
# ============================================================================

if len(results) > 0:
    comparison_df = pd.DataFrame(results).T
    comparison_df = comparison_df.drop(columns=['Exit_Reasons', 'Trend_Checks'], errors='ignore')
    comparison_df = comparison_df.sort_values('Total_Return_Pct', ascending=False)
    
    print('\n' + '=' * 70)
    print('INDIVIDUAL STOCK RESULTS')
    print('=' * 70)
    print(comparison_df.to_string())
    
    # Portfolio statistics
    print('\n' + '=' * 70)
    print('PORTFOLIO SUMMARY')
    print('=' * 70)
    
    total_stocks = len(comparison_df)
    stocks_with_trades = len(comparison_df[comparison_df['Total_Trades'] > 0])
    profitable_stocks = len(comparison_df[comparison_df['Total_Return_Pct'] > 0])
    
    print(f"\nStock Coverage:")
    print(f"  Stocks Tested:        {total_stocks}")
    print(f"  Stocks with Trades:   {stocks_with_trades} ({stocks_with_trades/total_stocks*100:.1f}%)")
    print(f"  Profitable Stocks:    {profitable_stocks} ({profitable_stocks/total_stocks*100:.1f}%)")
    
    avg_trades = comparison_df['Total_Trades'].mean()
    total_trades_all = comparison_df['Total_Trades'].sum()
    
    print(f"\nTrading Frequency:")
    print(f"  Total Trades:         {int(total_trades_all)}")
    print(f"  Avg Trades/Stock:     {avg_trades:.1f}")
    print(f"  Trades/Year/Stock:    {avg_trades/5:.1f}")
    
    active_stocks = comparison_df[comparison_df['Total_Trades'] > 0]
    if len(active_stocks) > 0:
        avg_win_rate = active_stocks['Win_Rate'].mean()
        avg_return = comparison_df['Total_Return_Pct'].mean()
        median_return = comparison_df['Total_Return_Pct'].median()
        avg_profit_factor = active_stocks[active_stocks['Profit_Factor'] != float('inf')]['Profit_Factor'].mean()
        
        print(f"\nPerformance Metrics:")
        print(f"  Avg Win Rate:         {avg_win_rate:.1f}%")
        print(f"  Avg Return:           {avg_return:+.1f}%")
        print(f"  Median Return:        {median_return:+.1f}%")
        print(f"  Avg Profit Factor:    {avg_profit_factor:.2f}")
        print(f"  Avg Drawdown:         {comparison_df['Max_Drawdown'].mean():.1f}%")
        print(f"  Avg Hold Time:        {comparison_df['Avg_Hold_Days'].mean():.1f} days")
        
        best_stock = comparison_df['Total_Return_Pct'].idxmax()
        worst_stock = comparison_df['Total_Return_Pct'].idxmin()
        
        print(f"\nTop Performers:")
        print(f"  Best:  {best_stock:6s} {comparison_df.loc[best_stock, 'Total_Return_Pct']:+.1f}%")
        print(f"  Worst: {worst_stock:6s} {comparison_df.loc[worst_stock, 'Total_Return_Pct']:+.1f}%")
        
        print(f"\nWin/Loss Analysis:")
        print(f"  Avg Win Size:         {comparison_df['Avg_Win_Pct'].mean():+.2f}%")
        print(f"  Avg Loss Size:        {comparison_df['Avg_Loss_Pct'].mean():.2f}%")
        if comparison_df['Avg_Loss_Pct'].mean() != 0:
            print(f"  Avg R:R Ratio:        {abs(comparison_df['Avg_Win_Pct'].mean() / comparison_df['Avg_Loss_Pct'].mean()):.2f}:1")
    
    # Combined trades analysis
    if len(trades) > 0:
        combined_trades = pd.concat(trades, ignore_index=True)
        
        print('\n' + '=' * 70)
        print('COMBINED TRADES ANALYSIS')
        print('=' * 70)
        
        winners = combined_trades[combined_trades['PnL'] > 0]
        losers = combined_trades[combined_trades['PnL'] < 0]
        
        print(f"\nTotal Trades: {len(combined_trades)}")
        print(f"  Winners: {len(winners)} ({len(winners)/len(combined_trades)*100:.1f}%)")
        print(f"  Losers:  {len(losers)} ({len(losers)/len(combined_trades)*100:.1f}%)")
        
        print(f"\nExit Reasons:")
        exit_summary = combined_trades['Exit_Reason'].value_counts()
        for reason, count in exit_summary.items():
            pct = count / len(combined_trades) * 100
            exit_trades = combined_trades[combined_trades['Exit_Reason'] == reason]
            avg_return = exit_trades['PnL_Pct'].mean()
            print(f"  {reason:12s}: {count:3d} ({pct:5.1f}%) | Avg: {avg_return:+.2f}%")
    
    # System evaluation
    print('\n' + '=' * 70)
    print('SYSTEM EVALUATION')
    print('=' * 70)
    
    is_viable = True
    issues = []
    
    if avg_trades < 2:
        issues.append("Too few trades (need 3+ per year)")
        is_viable = False
    
    if avg_win_rate < 45:
        issues.append("Win rate too low (need 45%+)")
        is_viable = False
    
    if avg_profit_factor < 1.3:
        issues.append("Profit factor too low (need 1.5+)")
        is_viable = False
    
    if avg_return < 5:
        issues.append("Average return too low (need 10%+)")
        is_viable = False
    
    if profitable_stocks / total_stocks < 0.5:
        issues.append("Too few profitable stocks (need 50%+)")
        is_viable = False
    
    if is_viable:
        print("✓ SYSTEM IS VIABLE!")
        print("\nStrengths:")
        if avg_win_rate >= 45:
            print(f"  ✓ Good win rate ({avg_win_rate:.1f}%)")
        if avg_profit_factor >= 1.5:
            print(f"  ✓ Strong profit factor ({avg_profit_factor:.2f})")
        if avg_trades >= 3:
            print(f"  ✓ Adequate trading frequency ({avg_trades:.1f} trades/year)")
        if avg_return >= 10:
            print(f"  ✓ Solid returns ({avg_return:+.1f}% avg)")
        if profitable_stocks / total_stocks >= 0.6:
            print(f"  ✓ Most stocks profitable ({profitable_stocks}/{total_stocks})")
        
        print("\nNext Steps:")
        print("  1. Test on different time periods (walk-forward)")
        print("  2. Focus on best-performing sectors")
        print("  3. Build position tracking for live trading")
        print("  4. Consider ML optimization for fine-tuning")
    else:
        print("⚠ SYSTEM NEEDS IMPROVEMENT")
        print("\nIssues Found:")
        for issue in issues:
            print(f"  ⚠ {issue}")
        
        print("\nRecommended Fixes:")
        if avg_trades < 2:
            print("  → Lower score threshold to 2 or relax R:R to 1.3")
        if avg_win_rate < 45:
            print("  → Strengthen trend filter or adjust entry timing")
        if avg_profit_factor < 1.3:
            print("  → Review stop/target distances")
        if avg_return < 5:
            print("  → Test on different ticker list or sectors")
    
    print('=' * 70)

else:
    print('\nNo results to analyze.')


print('\n' + '=' * 70)
print('TESTING DIFFERENT LISTS')
print('=' * 70)
print('\nTo test other stock lists, change this line in the code:')
print('  test_list = MIXED_QUALITY  # ← Change this')
print('\nAvailable lists:')
print('  - MIXED_QUALITY   (12 stocks - diversified, RECOMMENDED)')
print('  - HEALTHCARE      (5 stocks - telehealth focus)')
print('  - FINTECH         (5 stocks - financial tech)')
print('  - CLOUD_SAAS      (5 stocks - cloud software)')
print('  - CONSUMER        (5 stocks - e-commerce/apps)')
print('  - SEMIS           (5 stocks - semiconductor)')
print('  - OLD_CRYPTO_EV   (13 stocks - for comparison only)')

ENHANCED BACKTEST: Mixed Quality (Recommended)
Testing 1 stocks
Period: 2y
Trend Filter: ENABLED
Signal Thresholds: BUY (Score≥3, R:R≥1.5), STRONG BUY (Score≥5, R:R≥2.0)

[1/1] Testing HIMS...
Fetching HIMS data...
Got 502 days of data
Starting Enhanced Backtest with Trend Filter
Initial Capital: $10,000.00
Risk Per Trade: 1.0%
Signal Types: ['BUY', 'STRONG BUY']
Trend Filter: ENABLED (Strict)
Stop: 2.0× ATR | Target: 4.0× ATR
------------------------------------------------------------
  → 5 trades | 20.0% WR | -25.3% return | PF: 0.64
     Trend Filter: Passed 5, Rejected 19 setups

INDIVIDUAL STOCK RESULTS
     Total_Trades Winning_Trades Losing_Trades Win_Rate   Total_PnL Total_Return_Pct     Avg_Win   Avg_Loss Avg_Win_Pct Avg_Loss_Pct Profit_Factor Max_Drawdown Final_Capital Avg_Hold_Days
HIMS            5              1             4     20.0 -103.239031        -25.28919  185.627998 -72.216757   23.710914   -18.234561      0.642607   -25.331751   7471.080994          22.2

PORTFO